## Introduction
We would like to strengthen the linkages between invoices in the Chinook database.What tracks were involved (including the album and playlists in which they appear, as well as the artist who created the album), which clients purchased which tracks, and which staff members assisted which customers? will be the primary emphasis


## PostgreSQL to Neo4j Migration

### Exporting relational tables to CSV
```bash
COPY (SELECT * FROM "Artist") TO '/tmp/artists.csv' WITH CSV HEADER;
COPY (SELECT * FROM "Employee") TO '/tmp/employees.csv' WITH CSV HEADER;
COPY (SELECT * FROM "Customer") TO '/tmp/customers.csv' WITH CSV HEADER;
COPY (SELECT * FROM "Album") TO '/tmp/albums.csv' WITH CSV HEADER;
COPY (SELECT "Track".*, "Genre"."Name" AS "GenreName", "MediaType"."Name" AS "MediaTypeName" FROM "Track" LEFT OUTER JOIN "MediaType" ON "MediaType"."MediaTypeId" = "Track"."MediaTypeId" LEFT OUTER JOIN "Genre" ON "Genre"."GenreId" = "Track"."GenreId" ) TO '/tmp/tracks.csv' WITH CSV HEADER;
COPY (SELECT "Invoice".*, "InvoiceLine"."TrackId" AS "TrackId", "InvoiceLine"."InvoiceLineId" AS "InvoiceLineId" , "InvoiceLine"."UnitPrice" AS "UnitPrice", "InvoiceLine"."Quantity" AS "Quantity" FROM "Invoice" LEFT OUTER JOIN "InvoiceLine" ON "Invoice"."InvoiceId" =  "InvoiceLine"."InvoiceId") TO '/tmp/invoices.csv' WITH CSV HEADER;
COPY (SELECT "Playlist"."PlaylistId", "Name", "TrackId" FROM "Playlist" LEFT OUTER JOIN "PlaylistTrack" ON "Playlist"."PlaylistId" = "PlaylistTrack"."PlaylistId") TO '/tmp/playlists.csv' WITH CSV HEADER;
```
### Importing the data using Cypher
#### Creating the Artist nodes
Each row in the Artist table will be converted into a node labeled Artist. The cypher code below creates artist nodes with the properties artistId and name.

```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/artists.csv' AS row 
MERGE(artist:Artist {artistId: row.ArtistId}) 
    ON CREATE SET artist.name = row.Name
```
#### Creating the Album nodes
Each row in Album table will become a node with Album as the label. The cypher code below creates album nodes 

```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/albums.csv' AS row 
MERGE(album:Album {albumId: row.AlbumId}) 
    ON CREATE SET album.title = row.Title
```
#### Creating the Playlist nodes
Each row in Playlist will become a node with Playlist as the label. The cypher code below creates plalists nodes

```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/playlists.csv' AS row 
MERGE(playlist:Playlist {playlistId: row.PlaylistId}) 
    ON CREATE SET playlist.name = row.Name
```
#### Creating the Track nodes
Each row in Track table will become a node with Track as the label. The cypher code below creates track nodes for the composer property first it checks if it exists before setting the property

```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/tracks.csv' AS row 
MERGE(track:Track {trackId: row.TrackId, name: row.Name, milliseconds: row.Milliseconds, bytes: row.Bytes, unitPrice: row.UnitPrice, genre: row.GenreName, mediaType: row.MediaTypeName })
WITH track, row
WHERE row.composer IS NOT NULL AND row.composer <> ''
SET track.composer = row.Composer
```
#### Creating the Invoice nodes
Each row in Invoice table will become a node with Invoice as the label. The cypher code below creates invoice nodes. FOr the billing postal code we first check if it has a value before setting it

```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/invoices.csv' AS row 
MERGE(invoice:Invoice {invoiceId: row.InvoiceId, invoiceDate: row.InvoiceDate, billingAddress: row.BillingAddress, billingCity: row.BillingCity, billingCountry: row.BillingCountry, total: row.Total})
WITH invoice, row
WHERE row.billingPostalCode IS NOT NULL AND row.billingPostalCode <> ''
    SET invoice.billingPostalCode = row.billingPostalCode
```
### Creating the Employee nodes
Each row in Employee table will become a node with Employee as the label

```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/employees.csv' AS row 
MERGE(e:Employee {employeeId: row.EmployeeId, lastName: row.LastName, firstName: row.FirstName, title: row.Title, birthDate: row.BirthDate, hireDate: row.HireDate, address: row.Address, city: row.City, state: row.State, country: row.Country, postalCode: row.PostalCode, phone: row.Phone, fax: row.Fax, email: row.Email})
```

### Creating the Customer nodes
Each row in Customer table will become a node with Customer as the label. For all nullable columns the code checks if there is a value before setting it

```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/customers.csv' AS row 
MERGE(c:Customer {customerId: row.CustomerId, lastName: row.LastName, firstName: row.FirstName, address: row.Address, city: row.City, country: row.Country, email: row.Email})
WITH c, row
WHERE row.Company IS NOT NULL AND row.Company <> ''
    SET c.company = row.Company
WITH c, row
WHERE row.State IS NOT NULL AND row.State <> ''
    SET c.state = row.State
WITH c, row
WHERE row.Fax IS NOT NULL AND row.Fax <> ''
    SET c.fax = row.Fax
WITH c, row
WHERE row.PostalCode IS NOT NULL AND row.PostalCode <> ''
    SET c.postalCode = row.PostalCode
WITH c, row
WHERE row.Phone IS NOT NULL AND row.Phone <> ''
    SET c.phone = row.Phone
```
## Creating the indexes and constraints for the data
```cypher
CREATE INDEX employee_id FOR (e:Employee) ON (e.employeeId);
CREATE INDEX customer_id FOR (c:Customer) ON (c.customerId);
CREATE INDEX invoice_id FOR (i:Invoice) ON (i.invoiceId);
CREATE INDEX track_id FOR (t:Track) ON (t.trackId);
CREATE INDEX playlist_id FOR (p:Playlist) ON (p.playlistId);
CREATE INDEX album_id FOR (a:Album) ON (a.albumId);
CREATE INDEX artist_id FOR (a:Artist) ON (a.artistId);
```
## Creating the relationships between the nodes
### Creating the relationship between Artist and Album
Join between Artist and Album becomes a relationship named CREATED (were artist creates album)
```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/albums.csv' AS row
MATCH (album:Album {albumId: row.AlbumId})
MATCH (artist:Artist {artistId: row.ArtistId})
MERGE (artist)-[:CREATED]->(album)
```
### Creating the relationship between Track and Album
Join between Track and Album becomes a relationship named PART_OF (were track is part of album)
```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/tracks.csv' AS row
MATCH (album:Album {albumId: row.AlbumId})
MATCH (track:Track {trackId: row.TrackId})
MERGE (track)-[:PART_OF]->(album)
```

#### Creating the relationship between Customer and Invoice
Join between Customer and Invoice becomes a relationship named PURCHASED (were customer purchases an invoice)
```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/invoices.csv' AS row
MATCH (customer:Customer {customerId: row.CustomerId})
MATCH (invoice:Invoice {invoiceId: row.InvoiceId})
MERGE (customer)-[:PURCHASED]->(invoice)
```

### Creating the relationship between Employee and Customer
Join between Employee and Customer becomes a relationship named SUPPORTED_BY (were customer is supported by employee)

```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/customers.csv' AS row
MATCH (customer:Customer {customerId: row.CustomerId})
MATCH (employee:Employee {employeeId: row.SupportRepId})
MERGE (customer)-[:SUPPORTED_BY]->(employee)
```

### Creating the relationship between Employee and itself
Join between Employee and itself becomes a relationship named REPORTS_TO (were employee reports to another employee)
```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/employees.csv' AS row
MATCH (manager:Employee {employeeId: row.EmployeeId})
MATCH (employee:Employee {employeeId: row.ReportsTo})
MERGE (employee)-[:REPORTS_TO]->(manager)
```

### Creating the relationship between Playlist and Track
Join with join table (PlaylistTrack) will become a relationship named CONTAINS (were playlist contains tracks)
```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/playlists.csv' AS row
MATCH (p:Playlist {playlistId: row.PlaylistId})
MATCH (t:Track {trackId: row.TrackId})
MERGE (p)-[:CONTAINS]->(t)
```

### Creating the relationship between Track and Invoices
Join with join table (InvoiceLine) will become a relationship named SOLD_ON with properties unitPrice and quantity (were track is sold on invoice)
```cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ktxdev/AIM-5001/main/M13/1.%20Data/invoices.csv' AS row
MATCH (i:Invoice {invoiceId: row.InvoiceId})
MATCH (t:Track {trackId: row.TrackId})
MERGE (t)-[so:SOLD_ON]->(i)
    ON CREATE SET so.unitPrice = row.UnitPrice, so.quantity = row.Quantity
```


## Using Cypher to Retrieve Data from Neo4j


#### Task (a): Write and execute a Cypher query that returns all Tracks from the ‘Jazz’ genre composed by 'Miles Davis'
```cypher
MATCH (t:Track {genre: 'Jazz', composer: 'Miles Davis'}) 
RETURN t
```

#### Task (b): Write and execute a Cypher query that returns all Artists that have any Tracks available in the ‘AAC audio file’ media type

```cypher
MATCH (:Track {mediaType: 'AAC audio file'}) -[]- (:Album) -[] - (artist:Artist)
RETURN artist
```

#### Task (c): Write and execute a Cypher query that returns the Artist associated with the album ‘Bongo Fury’
```cypher
MATCH (:Album {title: 'Bongo Fury'} ) -[] - (artist:Artist)
RETURN artist
```

### TASK (d): Write and execute a Cypher query that returns all Tracks from the album ‘Coda’ by the artist ‘Led Zeppelin’
